# 전국어린이보호구역표준데이터
전국어린이보호구역표준데이터.csv

- 시설종류별 CCTV 설치 여부 카운트 / 대수
- 광역시도별, 시절종류별 카운트
- CCTV 평균 설치대수
- 관할 경찰서별 CCTV 대수 
- 도로폭 / CCTV 설치 대수 상관도

In [5]:
import numpy as np
import pandas as pd

pd_data = pd.read_csv('전국어린이보호구역표준데이터.csv', encoding='cp949')
drop_list = ['대상시설명', '소재지도로명주소', '소재지지번주소', '위도','경도','관리기관명','데이터기준일자','제공기관코드','제공기관명']
pd_data.drop(drop_list, axis=1, inplace=True)

#print(pd_data.shape)
# print(pd_data.dtypes)
# print(pd_data.head(3))


#index 찾기
for i in enumerate(pd_data.columns):
        print(i)

(0, '시설종류')
(1, '관할경찰서명')
(2, 'CCTV설치여부')
(3, 'CCTV설치대수')
(4, '보호구역도로폭')
